In [22]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.utils.data_utils import get_file
import tensorflow as tf
import numpy as np
import scipy.io
from tqdm import tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [23]:
# create a dictionary of annotations structures
# hashed by file name
# length of value is number of persons annotated
# [['person'],['gender'],['annotations_categories'],['annotations_continuous'],['gender'],['age']]
emotic_annotations = {}
def create_emotic_dict(data, test= False ): 
    # set variables based on testing/training dataset 
    # using combined avg on test dataset because 
    # multiple annotators used 
    if test: 
        categories = 'combined_categories'
        continuous  = 'combined_continuous'
    else: 
        categories = 'annotations_categories'
        continuous  = 'annotations_continuous'
    
    
    test_image = 'framesdb/images/frame_ghkq7yp0itqz0kn7.jpg'
    emotic_annotations = {} 
    
    print ("Creating body boxes")
    for i in range(len(data['filename'][0])): 
        file = data['filename'][0][i][0]
        # if more than one person is the focus of the image 
        for j in range(len(data['person'][0][i][0])):      
            file_path = str(data['folder'][0][i][0]) + "/" + file
            
            if file_path in emotic_annotations: 
                emotic_annotations[file_path] = np.append( emotic_annotations[file_path], 
                                                [[tuple(np.array(data['person'][0][i][0][j]['body_bbox'][0]).tolist()),
                                                  np.array(np.array(np.array(data['person'][0][i][0][j][categories]).tolist()).tolist()).flatten().tolist(),      
                                                  np.array(np.array(data['person'][0][i][0][j][continuous]).tolist()).flatten().tolist(),
                                                  np.array(data['person'][0][i][0][j]['gender']).tolist(),
                                                  np.array(data['person'][0][i][0][j]['age']).tolist()]], 
                                                  axis = 0 )

            else: 
                emotic_annotations[file_path]= np.array([[tuple(np.array(data['person'][0][i][0][j]['body_bbox'][0]).tolist()),
                                               np.array(np.array(np.array(data['person'][0][i][0][j][categories]).tolist()).tolist()).flatten().tolist(),                             
                                               np.array(np.array(data['person'][0][i][0][j][continuous]).tolist()).flatten().tolist(),
                                               np.array(data['person'][0][i][0][j]['gender']).tolist(),
                                               np.array(data['person'][0][i][0][j]['age']).tolist()]])
    
    return emotic_annotations




In [24]:
# load in emotic annotations
# Annotations = {train, val, test} 
# filename, folder, image_size, original_database, person 
# person: body_bbox, annotations_categories, annotations_continuous, gender, age
print ("Reading in emotic annotations")
emotic = scipy.io.loadmat('EMOTIC/annotations/Annotations.mat')

train_dict = create_emotic_dict(emotic['train'])
test_dict  = create_emotic_dict(emotic['test'], True)


Reading in emotic annotations
Creating body boxes
Creating body boxes


In [43]:
class GlobalFeatures():
    def __init__(self, files):
        
        self.features = self.extract_features(files)
        
    def extract_features(self,images): 
        # weights path to places365 dataset for location segmentation  
        WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels_notop.h5'
        model = VGG16()
        weights_path = get_file('vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                            WEIGHTS_PATH_NO_TOP,
                                            cache_subdir='models')

        # load pre-trained model minus top for feature extraction
        model = VGG16(weights= weights_path , include_top=False)
        
        dir_path = "EMOTIC/emotic/"
        features = [] 
        
        for i in tqdm(images): 
            img_path = dir_path + i
#             print(img_path)
#             img_path = '12.jpg'
            # load image setting the image size to 224 x 224
            img = image.load_img(img_path, target_size=(224, 224))
            # convert image to numpy array
            x = image.img_to_array(img)
            # the image is now in an array of shape (3, 224, 224) 
            # need to expand it to (1, 3, 224, 224) as preprocess takes in a list
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            # extract the features
            feats = model.predict(x)[0]
            # convert from Numpy to a list of values
            features = np.char.mod('%f', feats)

        return features
#         reduce dimensionality 
        


In [44]:
global_features = GlobalFeatures(train_dict)







  0%|          | 0/17077 [00:00<?, ?it/s]




  0%|          | 1/17077 [00:00<3:30:09,  1.35it/s]




  0%|          | 2/17077 [00:01<3:03:01,  1.55it/s]




  0%|          | 3/17077 [00:01<2:41:39,  1.76it/s]




  0%|          | 4/17077 [00:01<2:29:48,  1.90it/s]




  0%|          | 5/17077 [00:02<2:20:24,  2.03it/s]




  0%|          | 6/17077 [00:02<2:10:03,  2.19it/s]




  0%|          | 7/17077 [00:03<2:03:39,  2.30it/s]




  0%|          | 8/17077 [00:03<1:57:58,  2.41it/s]




  0%|          | 9/17077 [00:03<1:53:54,  2.50it/s]




  0%|          | 10/17077 [00:04<1:51:30,  2.55it/s]




  0%|          | 11/17077 [00:04<1:50:32,  2.57it/s]




  0%|          | 12/17077 [00:05<1:49:10,  2.61it/s]




  0%|          | 13/17077 [00:05<1:48:44,  2.62it/s]




  0%|          | 14/17077 [00:05<1:47:45,  2.64it/s]




  0%|          | 15/17077 [00:06<1:47:49,  2.64it/s]




  0%|          | 16/17077 [00:06<1:47:36,  2.64it/s]




  0%|          | 17/17077 [00:06<1:47:13,  2.6

  1%|          | 113/17077 [00:45<1:52:08,  2.52it/s]




  1%|          | 114/17077 [00:45<1:49:54,  2.57it/s]




  1%|          | 115/17077 [00:45<1:47:43,  2.62it/s]




  1%|          | 116/17077 [00:46<1:46:41,  2.65it/s]




  1%|          | 117/17077 [00:46<1:45:55,  2.67it/s]




  1%|          | 118/17077 [00:47<1:45:35,  2.68it/s]




  1%|          | 119/17077 [00:47<1:45:29,  2.68it/s]




  1%|          | 120/17077 [00:47<1:44:54,  2.69it/s]




  1%|          | 121/17077 [00:48<1:45:20,  2.68it/s]




  1%|          | 122/17077 [00:48<1:47:01,  2.64it/s]




  1%|          | 123/17077 [00:48<1:45:41,  2.67it/s]




  1%|          | 124/17077 [00:49<1:45:30,  2.68it/s]




  1%|          | 125/17077 [00:49<1:44:50,  2.70it/s]




  1%|          | 126/17077 [00:50<1:49:11,  2.59it/s]




  1%|          | 127/17077 [00:50<1:47:53,  2.62it/s]




  1%|          | 128/17077 [00:50<1:47:03,  2.64it/s]




  1%|          | 129/17077 [00:51<1:47:38,  2.62it/s]




  1%|         

  1%|▏         | 225/17077 [01:29<1:57:18,  2.39it/s]




  1%|▏         | 226/17077 [01:29<2:04:24,  2.26it/s]




  1%|▏         | 227/17077 [01:30<2:01:38,  2.31it/s]




  1%|▏         | 228/17077 [01:30<1:57:30,  2.39it/s]




  1%|▏         | 229/17077 [01:30<1:56:13,  2.42it/s]




  1%|▏         | 230/17077 [01:31<1:55:45,  2.43it/s]




  1%|▏         | 231/17077 [01:31<1:57:16,  2.39it/s]




  1%|▏         | 232/17077 [01:32<1:54:21,  2.46it/s]




  1%|▏         | 233/17077 [01:32<1:57:12,  2.40it/s]




  1%|▏         | 234/17077 [01:33<2:02:36,  2.29it/s]




  1%|▏         | 235/17077 [01:33<2:01:03,  2.32it/s]




  1%|▏         | 236/17077 [01:33<1:57:39,  2.39it/s]




  1%|▏         | 237/17077 [01:34<1:55:28,  2.43it/s]




  1%|▏         | 238/17077 [01:34<1:54:11,  2.46it/s]




  1%|▏         | 239/17077 [01:35<1:53:57,  2.46it/s]




  1%|▏         | 240/17077 [01:35<1:53:42,  2.47it/s]




  1%|▏         | 241/17077 [01:35<1:54:32,  2.45it/s]




  1%|▏        

  2%|▏         | 337/17077 [02:15<2:02:17,  2.28it/s]




  2%|▏         | 338/17077 [02:16<2:01:32,  2.30it/s]




  2%|▏         | 339/17077 [02:16<1:57:23,  2.38it/s]




  2%|▏         | 340/17077 [02:17<2:01:40,  2.29it/s]




  2%|▏         | 341/17077 [02:17<2:03:39,  2.26it/s]




  2%|▏         | 342/17077 [02:17<1:59:29,  2.33it/s]




  2%|▏         | 343/17077 [02:18<1:57:04,  2.38it/s]




  2%|▏         | 344/17077 [02:18<1:56:11,  2.40it/s]




  2%|▏         | 345/17077 [02:19<1:52:47,  2.47it/s]




  2%|▏         | 346/17077 [02:19<1:54:41,  2.43it/s]




  2%|▏         | 347/17077 [02:19<1:50:52,  2.51it/s]




  2%|▏         | 348/17077 [02:20<1:50:06,  2.53it/s]




  2%|▏         | 349/17077 [02:20<1:51:05,  2.51it/s]




  2%|▏         | 350/17077 [02:21<1:54:46,  2.43it/s]




  2%|▏         | 351/17077 [02:21<1:56:27,  2.39it/s]




  2%|▏         | 352/17077 [02:21<1:55:36,  2.41it/s]




  2%|▏         | 353/17077 [02:22<1:55:04,  2.42it/s]




  2%|▏        

  3%|▎         | 449/17077 [03:02<2:08:07,  2.16it/s]




  3%|▎         | 450/17077 [03:02<2:04:11,  2.23it/s]




  3%|▎         | 451/17077 [03:03<2:01:39,  2.28it/s]




  3%|▎         | 452/17077 [03:03<2:00:29,  2.30it/s]




  3%|▎         | 453/17077 [03:04<2:03:03,  2.25it/s]




  3%|▎         | 454/17077 [03:04<2:02:48,  2.26it/s]




  3%|▎         | 455/17077 [03:04<1:57:48,  2.35it/s]




  3%|▎         | 456/17077 [03:05<1:55:08,  2.41it/s]




  3%|▎         | 457/17077 [03:05<1:52:05,  2.47it/s]




  3%|▎         | 458/17077 [03:06<1:50:24,  2.51it/s]




  3%|▎         | 459/17077 [03:06<1:49:29,  2.53it/s]




  3%|▎         | 460/17077 [03:06<1:47:50,  2.57it/s]




  3%|▎         | 461/17077 [03:07<1:50:52,  2.50it/s]




  3%|▎         | 462/17077 [03:07<1:49:34,  2.53it/s]




  3%|▎         | 463/17077 [03:08<1:48:07,  2.56it/s]




  3%|▎         | 464/17077 [03:08<1:46:44,  2.59it/s]




  3%|▎         | 465/17077 [03:08<1:45:50,  2.62it/s]




  3%|▎        

...

 43%|████▎     | 7389/17077 [54:00<1:07:38,  2.39it/s]




 43%|████▎     | 7390/17077 [54:00<1:07:06,  2.41it/s]




 43%|████▎     | 7391/17077 [54:01<1:06:57,  2.41it/s]




 43%|████▎     | 7392/17077 [54:01<1:07:14,  2.40it/s]




 43%|████▎     | 7393/17077 [54:01<1:08:17,  2.36it/s]




 43%|████▎     | 7394/17077 [54:02<1:08:12,  2.37it/s]




 43%|████▎     | 7395/17077 [54:02<1:08:19,  2.36it/s]




 43%|████▎     | 7396/17077 [54:03<1:09:13,  2.33it/s]




 43%|████▎     | 7397/17077 [54:03<1:10:50,  2.28it/s]




 43%|████▎     | 7398/17077 [54:04<1:12:40,  2.22it/s]




 43%|████▎     | 7399/17077 [54:04<1:12:27,  2.23it/s]




 43%|████▎     | 7400/17077 [54:05<1:11:20,  2.26it/s]




 43%|████▎     | 7401/17077 [54:05<1:10:58,  2.27it/s]




 43%|████▎     | 7402/17077 [54:05<1:10:02,  2.30it/s]




 43%|████▎     | 7403/17077 [54:06<1:09:17,  2.33it/s]




 43%|████▎     | 7404/17077 [54:06<1:09:36,  2.32it/s]




 43%|████▎     | 7405/17077 [54:07<1:09:09,  2.33it/s]



 44%|████▍     | 7499/17077 [54:47<1:14:01,  2.16it/s]




 44%|████▍     | 7500/17077 [54:47<1:12:03,  2.22it/s]




 44%|████▍     | 7501/17077 [54:48<1:10:41,  2.26it/s]




 44%|████▍     | 7502/17077 [54:48<1:09:08,  2.31it/s]




 44%|████▍     | 7503/17077 [54:49<1:09:20,  2.30it/s]




 44%|████▍     | 7504/17077 [54:49<1:08:40,  2.32it/s]




 44%|████▍     | 7505/17077 [54:49<1:07:13,  2.37it/s]




 44%|████▍     | 7506/17077 [54:50<1:07:00,  2.38it/s]




 44%|████▍     | 7507/17077 [54:50<1:06:51,  2.39it/s]




 44%|████▍     | 7508/17077 [54:51<1:06:46,  2.39it/s]




 44%|████▍     | 7509/17077 [54:51<1:06:32,  2.40it/s]




 44%|████▍     | 7510/17077 [54:51<1:06:48,  2.39it/s]




 44%|████▍     | 7511/17077 [54:52<1:09:27,  2.30it/s]




 44%|████▍     | 7512/17077 [54:52<1:08:31,  2.33it/s]




 44%|████▍     | 7513/17077 [54:53<1:08:20,  2.33it/s]




 44%|████▍     | 7514/17077 [54:53<1:20:31,  1.98it/s]




 44%|████▍     | 7515/17077 [54:54<1:18:42,  2.02it/s]



 45%|████▍     | 7609/17077 [55:37<1:08:32,  2.30it/s]




 45%|████▍     | 7610/17077 [55:38<1:14:01,  2.13it/s]




 45%|████▍     | 7611/17077 [55:38<1:17:52,  2.03it/s]




 45%|████▍     | 7612/17077 [55:39<1:14:29,  2.12it/s]




 45%|████▍     | 7613/17077 [55:39<1:12:00,  2.19it/s]




 45%|████▍     | 7614/17077 [55:40<1:09:52,  2.26it/s]




 45%|████▍     | 7615/17077 [55:40<1:08:35,  2.30it/s]




 45%|████▍     | 7616/17077 [55:40<1:08:03,  2.32it/s]




 45%|████▍     | 7617/17077 [55:41<1:07:16,  2.34it/s]




 45%|████▍     | 7618/17077 [55:41<1:06:43,  2.36it/s]




 45%|████▍     | 7619/17077 [55:42<1:06:07,  2.38it/s]




 45%|████▍     | 7620/17077 [55:42<1:05:32,  2.40it/s]




 45%|████▍     | 7621/17077 [55:42<1:05:19,  2.41it/s]




 45%|████▍     | 7622/17077 [55:43<1:09:17,  2.27it/s]




 45%|████▍     | 7623/17077 [55:43<1:10:41,  2.23it/s]




 45%|████▍     | 7624/17077 [55:44<1:09:36,  2.26it/s]




 45%|████▍     | 7625/17077 [55:44<1:08:17,  2.31it/s]



 45%|████▌     | 7719/17077 [56:25<1:06:32,  2.34it/s]




 45%|████▌     | 7720/17077 [56:26<1:05:55,  2.37it/s]




 45%|████▌     | 7721/17077 [56:26<1:06:17,  2.35it/s]




 45%|████▌     | 7722/17077 [56:26<1:06:25,  2.35it/s]




 45%|████▌     | 7723/17077 [56:27<1:11:25,  2.18it/s]




 45%|████▌     | 7724/17077 [56:27<1:10:45,  2.20it/s]




 45%|████▌     | 7725/17077 [56:28<1:09:51,  2.23it/s]




 45%|████▌     | 7726/17077 [56:29<1:22:57,  1.88it/s]




 45%|████▌     | 7727/17077 [56:29<1:18:46,  1.98it/s]




 45%|████▌     | 7728/17077 [56:29<1:15:13,  2.07it/s]




 45%|████▌     | 7729/17077 [56:30<1:17:44,  2.00it/s]




 45%|████▌     | 7730/17077 [56:30<1:14:41,  2.09it/s]




 45%|████▌     | 7731/17077 [56:31<1:13:29,  2.12it/s]




 45%|████▌     | 7732/17077 [56:31<1:12:35,  2.15it/s]




 45%|████▌     | 7733/17077 [56:32<1:12:46,  2.14it/s]




 45%|████▌     | 7734/17077 [56:32<1:12:40,  2.14it/s]




 45%|████▌     | 7735/17077 [56:33<1:10:55,  2.20it/s]



KeyboardInterrupt: 